In [12]:
import humanize
import pandas as pd
from pathlib import Path

In [13]:
collected = [
    pd.read_csv(csv).rename(columns=lambda _: _.strip())
    for csv in sorted(Path('dict-metrics').glob('*.csv'))
]

In [14]:
collected[0].describe()

,usage count,total time,extra 1,extra 2,extra 3
count,164.000000,1.640000e+02,164.000000,164.000000,164.000000
mean,6999.975610,6.918620e+05,1907.414634,2568.060976,5.018293
std,32837.598097,2.043899e+06,13181.706649,16284.980741,64.265503
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,65.500000,1.431450e+04,0.000000,0.000000,0.000000
75%,964.000000,2.815492e+05,0.000000,0.000000,0.000000
max,305023.000000,1.670231e+07,156977.000000,148046.000000,823.000000


In [15]:
over_1_mil = []

for coll in collected:
    if coll['usage count'].max() > 1_000_000:
        over_1_mil.append(coll)

In [16]:
print(f"{len(collected)=} {len(over_1_mil)=}")

len(collected)=5795 len(over_1_mil)=2113


In [17]:
over_1_mil_df = pd.concat(over_1_mil).groupby(['method']).sum().reset_index('method')
over_1_mil_df

,method,usage count,total time,extra 1,extra 2,extra 3
0,PyDict_Clear,4205669,24793128045,0,0,0
1,PyDict_Contains,11472628641,2730593362007,228695482,11243933159,0
2,PyDict_Copy,13662633,4758857536,0,0,0
3,PyDict_DelItem,144483576,69649565616,0,0,0
4,PyDict_DelItemString,28835,42261501,0,0,0
...,...,...,...,...,...,...
159,new_values,122307211,5949903179,0,0,0
160,shared_keys_usable_size,176907270,3073671210,49974,0,0
161,unicode_get_hash,29933118453,521522896847,0,0,0
162,unicodekeys_lookup_generic,9681638,617332378,67684,9613954,0


In [18]:
running_time = (over_1_mil_df[['total time']].sum() / 10**9)[['total time']].iloc[0]
humanize.naturaltime(running_time)

'3 hours ago'

In [19]:
over_1_mil_df['avg running time'] = over_1_mil_df['total time'] / over_1_mil_df['usage count']
over_1_mil_df

,method,usage count,total time,extra 1,extra 2,extra 3,avg running time
0,PyDict_Clear,4205669,24793128045,0,0,0,5895.168651
1,PyDict_Contains,11472628641,2730593362007,228695482,11243933159,0,238.009392
2,PyDict_Copy,13662633,4758857536,0,0,0,348.311891
3,PyDict_DelItem,144483576,69649565616,0,0,0,482.058706
4,PyDict_DelItemString,28835,42261501,0,0,0,1465.632079
...,...,...,...,...,...,...,...
159,new_values,122307211,5949903179,0,0,0,48.647199
160,shared_keys_usable_size,176907270,3073671210,49974,0,0,17.374477
161,unicode_get_hash,29933118453,521522896847,0,0,0,17.422939
162,unicodekeys_lookup_generic,9681638,617332378,67684,9613954,0,63.763216


In [20]:
unused = over_1_mil_df.loc[over_1_mil_df['usage count'] == 0, "method"]
unused

19                PyDict_Values
30     _PyDict_DebugMallocStats
31            _PyDict_DelItemId
48          _PyDict_NewPresized
66             all_contained_in
74       dict___reversed___impl
91                      dict_or
104                 dict_values
106          dictitems_contains
109          dictitems_reversed
110               dictitems_xor
117             dictiter_reduce
126           dictkeys_reversed
129        dictreviter_iternext
132         dictvalues_reversed
135            dictview_mapping
136               dictview_repr
137        dictview_richcompare
139        dictviews_isdisjoint
143               dictviews_xor
Name: method, dtype: object

In [21]:
over_1_mil_df.to_csv('dict-metrics_combined.csv', index=False)

In [22]:
lookups = [  # aggregate
    "_Py_dict_lookup",
    ""
]